In [ ]:
from typing import TypedDict, List, Annotated
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import os

In [ ]:
# 환경 변수 설정
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"


# FAISS 벡터 DB 불러오기
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")
vectorstore = FAISS.load_local("card_QA_faiss_db", embedding_model,allow_dangerous_deserialization=True)


In [ ]:
# 1. State 정의
class GraphState(TypedDict):
    question: Annotated[str, "질문"]
    answer: Annotated[str, "답변"]
    score: Annotated[float, "유사도 점수"]
    retriever_docs: Annotated[List[Document], "유사도 상위문서"]

# 2. retriever_node 정의
def retriever_node(state: GraphState) -> GraphState:
    docs = vectorstore.similarity_search_with_score(state["question"], k=3)
    retrieved_docs = [doc for doc, _ in docs]
    score = docs[0][1]
    # print("\n[retriever_node] 문서:", [doc.page_content for doc in retrieved_docs])
    # print("[retriever_node] 유사도 점수:", score)
    # print('-'*100)
    # print(docs)
    # print('-'*100)
    # print(retrieved_docs[0])
    return GraphState(score=score, retriever_docs=retrieved_docs)